<a href="https://colab.research.google.com/github/boopatt/BookshelfProject/blob/main/HandlingDataSkew_DF_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Distributed Processing Challenges: Handling Data Skew in DF PySpark**

**`Udemy Course: Best Hands-on Big Data Practices and Use Cases using PySpark`**

**`Author: Amin Karami (PhD, FHEA)`**



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load Spark engine
!pip3 install -q findspark
import findspark
findspark.init()

# import SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("SkewinDF").getOrCreate()

spark

# (1) Loading Data Skew

To understand skew, we create a random data where keys are uniformly distributed.

In [ ]:
import numpy as np
import pandas as pd
import random


# sale dataset:
# table 1: OrderID, Qty, Sales, Discount (yes=1, no=0)
# table 2: ProductID, OrderID, Product, Price

########### Table 1 ###########

key_1 = [101] * 100
key_2 = [201] * 7000000
key_3 = [301] * 500
key_4 = [401] * 10000
OrderID = key_1 + key_2 + key_3 + key_4
random.shuffle(OrderID)

Qty_1 = list(np.random.randint(low = 1, high = 100, size = len(key_1)))
Qty_2 = list(np.random.randint(low = 1, high = 200, size = len(key_2)))
Qty_3 = list(np.random.randint(low = 1, high = 1000, size = len(key_3)))
Qty_4 = list(np.random.randint(low = 1, high = 50, size = len(key_4)))
Qty = Qty_1 + Qty_2 + Qty_3 + Qty_4

Sales_1 = list(np.random.randint(low = 10, high = 100, size = len(key_1)))
Sales_2 = list(np.random.randint(low = 50, high = 3400, size = len(key_2)))
Sales_3 = list(np.random.randint(low = 12, high = 2000, size = len(key_3)))
Sales_4 = list(np.random.randint(low = 40, high = 1000, size = len(key_4)))
Sales = Sales_1 + Sales_2 + Sales_3 + Sales_4

Discount = list(np.random.randint(low = 0, high = 2, size = len(OrderID)))
data1 = list(zip(OrderID,Qty,Sales,Discount))

# Create the Pandas DF
data_skew = pd.DataFrame(data1, columns=['OrderID','Qty','Sales','Discount'])


########### Table 2 ###########
data2 = [[1, 101, 'pencil', 4.99],
         [2, 101, 'book', 9.5],
         [3, 101, 'scissors', 14],
         [4, 301, 'glue', 7],
         [5, 201, 'marker', 8.49],
         [6, 301, 'label', 2],
         [7, 201, 'calculator', 3.99],
         [8, 501, 'eraser', 1.55],
        ]

data_small = pd.DataFrame(data2, columns=['ProductID', 'OrderID', 'Product', 'Price'])

In [ ]:
# Create PySpark DF from Pandas

# Optimize conversion between PySpark and Pandas DF: Enable arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

df_skew = spark.createDataFrame(data_skew)
df_skew.printSchema()
df_skew.show()
df_skew.rdd.getNumPartitions()

In [ ]:
df_small = spark.createDataFrame(data_small)
df_small.printSchema()
df_small.show()
df_small.rdd.getNumPartitions()

# (2) Run a shuffle `Join()` with small sized data

## Mitigate data skewness: SALTING

# (3) Run a shuffle `Join()` to see how the skew effects computation resources.

## Mitigate data skewness: SALTING

## Mitigate data skewness: Broadcast Hash Join